In [36]:
 # Initial imports
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from pathlib import Path
import numpy as np 
import pandas as pd

In [37]:
# Read in data
data = Path("tsla_cleansed.csv")
df = pd.read_csv(data)
df.head()

,date,compound,negative,neutral,positive,text,Prices
0,2012-05-23,0.9972,0.067,0.737,0.197,In today s bubble 2 0 world billions of VC do...,0.007470
1,2012-06-21,0.6808,0.067,0.836,0.097,EquitiesAsian markets gained on Wednesday ahe...,-0.048239
2,2012-08-05,0.9971,0.034,0.822,0.144,Entrepreneurs that have the capacity to change...,NaN
3,2012-10-19,0.9898,0.069,0.781,0.150,Google Inc GOOG got caught naked yesterday ...,-0.010734
4,2012-11-19,0.9893,0.006,0.917,0.077,Recent Performance of the Stocks Mentioned Bel...,0.031387


In [38]:
df_new = df.drop(columns = ["text","date"] )

In [39]:
df_new.dropna(inplace=True)

In [40]:
df_new

,compound,negative,neutral,positive,Prices
0,0.9972,0.067,0.737,0.197,0.007470
1,0.6808,0.067,0.836,0.097,-0.048239
3,0.9898,0.069,0.781,0.150,-0.010734
4,0.9893,0.006,0.917,0.077,0.031387
5,0.2023,0.059,0.879,0.062,0.033593
...,...,...,...,...,...
4278,0.9910,0.013,0.890,0.096,0.103150
4279,0.9960,0.068,0.824,0.108,0.103150
4280,0.9859,0.044,0.846,0.111,0.103150
4281,0.9722,0.086,0.804,0.109,0.103150


In [41]:
 # Define the features X set and the target y vector
X = df_new.iloc[:, 0:4].values
y = df_new["Prices"].values

In [42]:
# Scale the data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X)
X = scaler.transform(X)

In [43]:
# Define the model - shallow neural network
nn = Sequential()

# Hidden layer
nn.add(Dense(units=4, input_dim=4, activation="relu"))

# Output layer
nn.add(Dense(units=1, activation="linear"))

In [44]:
# Compile the model
nn.compile(loss="mean_squared_error", optimizer="adam", metrics=["mse"])

# Fit the model
model = nn.fit(X, y, epochs=200, verbose=0)

In [45]:
# Save model as JSON
nn_json = nn.to_json()

file_path = Path("model_1.json")
with open(file_path, "w") as json_file:
    json_file.write(nn_json)

# Save weights
file_path = "model_1.h5"
nn.save_weights("model_1.h5")

In [84]:
# Read in data
data2 = Path("new_tesla_news.csv")
df2 = pd.read_csv(data2)
df2.head()

,date,compound,negative,neutral,positive,text
0,2021-01-02,0.3182,0.00,0.937,0.063,"Tesla delivered 499,550 vehicles in 2020, a 36..."
1,2021-01-14,0.4215,0.00,0.922,0.078,"However, the feds, in the form of NHTSA, aren’..."
2,2020-12-31,0.2732,0.05,0.840,0.109,Tesla has launched the world’s largest Superch...
3,2020-12-29,0.4215,0.00,0.915,0.085,An Apple spokesperson told the publication tha...
4,2020-12-26,0.6369,0.00,0.864,0.136,Your Tesla could soon broadcast your taste in ...


In [85]:
df_new3 = df2.drop(columns = ["text","date"] )

In [86]:
df_new3.dropna(inplace=True)

In [87]:
#df_new2['Prices'] = np.nan

In [88]:
df_new3.head()

,compound,negative,neutral,positive
0,0.3182,0.00,0.937,0.063
1,0.4215,0.00,0.922,0.078
2,0.2732,0.05,0.840,0.109
3,0.4215,0.00,0.915,0.085
4,0.6369,0.00,0.864,0.136


In [89]:
 # Define the features X set and the target y vector
X2 = df_new3.iloc[:, 0:4].values

In [90]:
# Scale the data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X2)
X2 = scaler.transform(X2)

In [91]:
# Load the saved model to make predictions
from tensorflow.keras.models import model_from_json

# load json and create model
file_path = Path("model_1.json")
with open(file_path, "r") as json_file:
    model_json = json_file.read()
loaded_model = model_from_json(model_json)

# load weights into new model
file_path = "model_1.h5"
loaded_model.load_weights(file_path)

In [92]:
# Make some predictions with the loaded model
df_new3["predicted"] = loaded_model.predict(X2)
df_new3.head(100)

,compound,negative,neutral,positive,predicted
0,0.3182,0.000,0.937,0.063,-0.000087
1,0.4215,0.000,0.922,0.078,-0.000087
2,0.2732,0.050,0.840,0.109,-0.000087
3,0.4215,0.000,0.915,0.085,-0.000087
4,0.6369,0.000,0.864,0.136,-0.000087
5,0.0000,0.000,1.000,0.000,-0.000087
6,-0.0516,0.065,0.876,0.059,-0.001529
7,0.6124,0.000,0.872,0.128,-0.000087
8,0.3612,0.000,0.928,0.072,-0.000087
9,0.0000,0.000,1.000,0.000,-0.000087


In [95]:
mean = df_new3['predicted'].mean()

In [96]:
mean

-0.001249351305887103

In [97]:
today_price = 850

In [104]:
future_price = today_price + (today_price * mean)
print (f"future price = {future_price}")

future price = 848.938051389996
